# Step 1: Load Environment Variables

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Retrieve the API key
groq_api_key = os.getenv("GROQ_API_KEY")


# Step 2: Initialize the Model

In [2]:
from langchain_groq import ChatGroq

# Initialize the ChatGroq model
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)


# Step 3: Set Up Session History Store

In [3]:
from langchain_community.chat_message_histories import ChatMessageHistory # ChatMessageHistory is used to manage chat message histories in a session.
from langchain_core.chat_history import BaseChatMessageHistory  # BaseChatMessageHistory serves as a base class for all chat message history management classes.
from langchain_core.runnables.history import RunnableWithMessageHistory # RunnableWithMessageHistory wraps a runnable (e.g., a model) with the capability to manage its message history.




store = {}  # Dictionary to store session-specific chat histories


# Function to retrieve or initialize the chat history for a specific session
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    
    if session_id not in store:  # Check if the session ID exists in the store
        store[session_id] = ChatMessageHistory()  # If not, initialize a new ChatMessageHistory object and assign it to the session ID
    return store[session_id]  # Return the chat message history object for the session




# Wrapping the model with message history functionality.This allows the model to maintain and access session-specific chat histories.
with_message_history = RunnableWithMessageHistory(model, get_session_history)


# Step 4: Create a Prompt Template

In [4]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Respons in Such a way that you are a {role}.",  # Rule
            
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

# Step 5: Wrap the Model with Session-Specific History

In [5]:
from langchain_core.runnables.history import RunnableWithMessageHistory

with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

# Step 6: Example Usage

In [10]:
from langchain_core.messages import HumanMessage
config = {"configurable": {"session_id": "chat1"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Nahid")],"role":"English to Bangla Tanslator"},
    config=config
)
repsonse.content

'নমস্কার, নাহিদ! 👋\n\n(Hello, Nahid!) \n\nWhat can I translate for you? 😊  \n\n'

In [11]:
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="I am From Bangladesh")],"role":"English to Bangla Tanslator"},
    config=config
)
repsonse.content

"আপনি বাংলাদেশ থেকে আসছেন!  🇧🇩\n\n(You are from Bangladesh!)  \n\n  Welcome! Is there anything you'd like me to translate?  \n\n\n\n"

In [12]:
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="I am From Noakhali Science And Technology University")],"role":"English to Bangla Tanslator"},
    config=config
)
repsonse.content

'আপনি নোয়াখালী বিজ্ঞান ও প্রযুক্তি বিশ্ববিদ্যালয় থেকে!  🏫\n\n(You are from Noakhali Science and Technology University!)\n\n  \n\nWhat can I help you translate today? 😊\n'

In [13]:
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="what do you know about me?")],"role":"English to Bangla Tanslator"},
    config=config
)
repsonse.content

"From what you've told me, I know:\n\n* **Your name is Nahid.**\n* **You are from Bangladesh.**\n* **You are a student at Noakhali Science and Technology University.**\n\nIs there anything else you'd like to share? 😊  I'm here to learn more! \n\n\n\n\n"